# BELUM TAMBAH KALO ADA OUTLIER

In [1]:
import time
start = time.time()

In [2]:
import pandas as pd
import random
import re
import math
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()


In [3]:
#this function does cleaning, tokenize, remove stopwords, and stemming
def get_clean_corpus(corpus, stopwords):
    
    #segmentasi
    temp = sent_tokenize(corpus)
    corpus = pd.DataFrame(temp, columns=['dokumen'])
    
    clean_corpus = []
    token = []
    for index, sentence in enumerate(corpus['dokumen']):
        term = word_tokenize(corpus['dokumen'][index])
        
        #deleting url
        deleted_url = [temp for temp in term if not re.match(r"\w+(?:(\.(\w+)\.(\w+)))|\w+(?:(\.(\w+)))", str(temp))]
        
        #deleting symbol
        deleted_symbol = [re.sub(r"[\-\+\=\:\;\"\\\@\[\]\,_!;.':#$%^&*()<>?/\|}{~:]"," ",str(temp)) for temp in deleted_url ]
        
#         print(" ".join(deleted_symbol))
        #stemming
        stemmed_sentence = stemmer.stem(" ".join(deleted_symbol))
#         print(stemmed_sentence)
        
        tokens = word_tokenize(stemmed_sentence)
        
        for i in range(len(tokens)):
            for index, word in enumerate(tokens):
                #delete stopwprds
                if word in stopwords:
                    del tokens[index]
                    
                #delete number
                if word.isdigit():
                    del tokens[index]
        
        clean_corpus.append(" ".join(tokens))  
        token.append(list(dict.fromkeys(tokens)))
        
    corpus['clean_corpus'] = clean_corpus
    corpus['terms'] = token
    
    return corpus

In [4]:
input_document = open("bbc contoh beneran.txt", "r")
document= input_document.readline()

In [5]:
stopword = open("stopword_list_tala.txt", "r")
stopwords = stopword.read().split("\n")

In [6]:
cleaning_result = get_clean_corpus(corpus=document, stopwords=stopwords)

In [7]:
cleaning_result

,dokumen,clean_corpus,terms
0,"Sejumlah sekolah di Kota Palembang, Sumatera S...",sekolah kota palembang sumatera selatan putus ...,"[sekolah, kota, palembang, sumatera, selatan, ..."
1,"Kepala SMP Negeri 7 Palembang, Siti Zubaida, m...",kepala smp negeri palembang siti zubaida putus...,"[kepala, smp, negeri, palembang, siti, zubaida..."
2,"""Pagi ini kami memulangkan siswa karena meliha...",pagi pulang siswa lihat kabut asap tebal dampa...,"[pagi, pulang, siswa, lihat, kabut, asap, teba..."
3,Hal ini diamini Kepala Dinas Pendidikan Kota P...,amin kepala dinas didik kota palembang ahmad z...,"[amin, kepala, dinas, didik, kota, palembang, ..."
4,"""Hari ini seluruh TK hingga SMP negeri dan swa...",tk smp negeri swasta derajat libur besok edar ...,"[tk, smp, negeri, swasta, derajat, libur, beso..."
5,"Menurutnya, kalau kualitas udara Palembang mas...",kualitas udara palembang buruk giat ajar ajar ...,"[kualitas, udara, palembang, buruk, giat, ajar..."
6,"Akan tetapi, sebagaimana dipaparkan Kepala Din...",papar kepala dinas didik sumatera selatan wido...,"[papar, kepala, dinas, didik, sumatera, selata..."
7,"""Daerah yang tidak terdampak kabut asap tetap ...",daerah dampak kabut asap normal ajar daerah ka...,"[daerah, dampak, kabut, asap, normal, ajar, ka..."
8,"Hal itu belakangan dibenarkan Agus Wibowo, sel...",agus wibowo kepala pusat data informasi humas ...,"[agus, wibowo, kepala, pusat, data, informasi,..."
9,"""Melalui pesan digital, Kepala Dinas Pendidika...",pesan digital kepala dinas didik kota palemban...,"[pesan, digital, kepala, dinas, didik, kota, p..."


In [8]:
#this function calculates term weighting
def get_term_weighting_score(cleaning_result):
    
    #getting all the terms
    terms = []
    for index, sentence in enumerate(cleaning_result['terms']):
        terms += [temp for temp in sentence if temp not in terms]
    terms.sort()
        
    #getting frequency for every sentences
    terms_frequency = pd.DataFrame()
    for index, term in enumerate(cleaning_result['terms']):
        frequency_each_sentence = []
        for i, d in enumerate(terms):
            temp = term.count(d)
            frequency_each_sentence.append(temp)
        terms_frequency[str(index+1)] = frequency_each_sentence
        
    terms_frequency['terms'] = terms
    terms_frequency.set_index('terms', inplace= True)
    
    #getting df for every terms
    df_idf = pd.DataFrame(terms_frequency.sum(axis=1), columns=['df_term'])
    df_idf['terms'] = terms
    df_idf.set_index('terms', inplace= True)
    
    #getting idf for every terms
    N = len(terms_frequency.columns)
    terms_idf = []
    for i, d in df_idf.iterrows():
        idf_score = math.log((N+1)/((df_idf['df_term'][i])), 10)
        terms_idf.append(idf_score)
    df_idf['idf_term'] = terms_idf
    
    return terms_frequency, df_idf   

In [9]:
terms_frequency, df_idf  = get_term_weighting_score(cleaning_result=cleaning_result)

In [10]:
# terms_frequency
# df_idf

In [11]:
class Sentence:
    np.random.seed(0)
    def __init__(self, id, full_sentence, clean_sentence, tokens):
        self.list_bm25 = {}
        self.pagerank_score = random.random()
#         self.pagerank_score = pagerank
        self.id = id
        self.full_sentence = full_sentence
        self.clean_sentence = clean_sentence
        self.tokens = tokens
        self.sentence_len = len(clean_sentence.split())
        self.pagerank_score_new = 0
            
    def calculate_bm25(self, raw_frequency, idf, doc, slen_ave):
        k1 = 1.2
        b = 0.75
        total_bm25 = 0
        for query in self.tokens:
            tf = raw_frequency.at[str(query),str(doc.id)]
            idff = idf.at[str(query),'idf_term']
            temp = idff * ((k1+1) * tf) / (k1*( (1-b) + (b *(self.sentence_len/slen_ave)) ) + tf)            
            total_bm25 += temp
        self.list_bm25[doc.id] = total_bm25
        
    def calculate_new_pagerank(self, doc):
        d=0.85
        sum_InVi = 0
        for item in doc:
            if self.id is not item.id:
                Wji = self.list_bm25[item.id]
                total_Wjk = sum(item.list_bm25.values())
                sum_InVi += Wji/total_Wjk*item.pagerank_score
        self.pagerank_score_new = (1-d)+(d*sum_InVi)
#         print(self.pagerank_score_new)
                

In [12]:
class Graph:    
    def __init__(self, result_doc, raw_frequency, idf):
        self.raw_frequency = raw_frequency
        self.idf = idf.drop(columns=['df_term'])
        self.result_doc = result_doc
        self.total_doc = len(result_doc['dokumen'])
        self.slen_ave = 0
        self.summarize = []
        self.doc = []
        self.outlier = []
        
        
        #making object sentence
        list_pgrk = [0.400827866,0.863170087,0.389187762,0.924094751,0.157640608,0.714980958,0.216858534,0.237221536,0.076112858,0.841401681]

#         doc = []
        for index, item in self.result_doc.iterrows():
            self.doc.append(Sentence((index+1), item['dokumen'], item['clean_corpus'], item['terms'])) 
        
        #calculate len average
        temp_len_doc = 0
        for item in self.doc:
            temp_len_doc += item.sentence_len
        self.slen_ave = temp_len_doc/len(self.doc)
        
        
        #calculate bm25 for each object sentence
        for item in self.doc:
            for item2 in self.doc:
                if item.id is not item2.id:
                    item.calculate_bm25(raw_frequency= self.raw_frequency, idf=self.idf, doc=item2, slen_ave=self.slen_ave)
           
        
        ##CHECKING IF BM25 SCORE IS 0 (OUTLIER SENTENCE)
        for index, item in enumerate(self.doc):
#             print(sum(item.list_bm25.values()))
            if sum(item.list_bm25.values()) <= 0:
                self.outlier.append(self.doc.pop(index))
                
                
        #calculate pagerank
        for i in range(100):
            for item in self.doc:
                item.calculate_new_pagerank(self.doc)
            
            #update pagerank score
            for item in self.doc:
                item.pagerank_score = item.pagerank_score_new
        

        #getting the summarize        
        sorted_doc = sorted(self.doc, key=lambda x: x.pagerank_score, reverse=True)        
        top_pagerank = []
        for item in range(math.ceil(self.total_doc*0.30)):
            top_pagerank.append(sorted_doc[item])
        
        sorted_sum = sorted(top_pagerank, key=lambda x: x.id)
        
        temp_summarize = [item.full_sentence for item in sorted_sum]
        
        self.summarize = sorted_sum
#         self.summarize = temp_summarize
   

In [13]:
cobs = Graph(result_doc=cleaning_result, raw_frequency=terms_frequency, idf=df_idf)
for item in cobs.summarize:
    print(item.id)
    print(item.full_sentence)
#     print(item.pagerank_score)

2
Kepala SMP Negeri 7 Palembang, Siti Zubaida, mengatakan keputusan pemulangan ditempuh sesuai dengan instruksi Dinas Pendidikan Kota Palembang.
7
Akan tetapi, sebagaimana dipaparkan Kepala Dinas Pendidikan Sumatera Selatan, Widodo, kegiatan belajar mengajar di daerah yang tidak terdampak kabut asap tetap berlangsung.
10
"Melalui pesan digital, Kepala Dinas Pendidikan Kota Palembang menginstruksikan kegiatan belajar mengajar di tingkat paud, TK, SD dan SMP negeri dan swasta diliburkan hingga batas yang belum ditentukan," sebut Agus dalam siaran pers.


In [14]:
coba = "Indonesia adalah salah satu Negara yang di dalamnya memiliki iklim tropis. Negara ini terdiri dari banyak pulau di setiap wilayahnya. Meskipun harus diakui bahwa di Indonesia sendiri, dataran yang ada tentu tidak seluas lautan yang membentang luas. Namun demikian, Indonesia memiliki kawasan hutan yang cukup banyak mulai dari Sabang yang terletak di provinsi Aceh sampai Merauke yang terletak di kawasan Papua. Namun, di beberapa tahun terakhir, Indonesia sering mengalami kebakaran hutan lantaran beberapa faktor yang ada, mulai dari faktor buatan atau dari manusianya sendiri hingga faktor alam. Faktor alam yang menyebabkan kebakaran hutan memang tidak bisa dihindarkan dan tidak ada yang bisa disalahkan dalam hal ini. Akan tetapi, untuk faktor dari tindakan manusia perlu untuk ditindak dan dievaluasi. Memang sebuah keresahan tersendiri dimana manusia banyak yang kini kehilangan kesadarannya sampai-sampai melakukan suatu perbuatan yang bisa merugikan banyak orang termasuk dirinya sendiri, khususnya merugikan lingkungan hidup. Sedangkan hutan sendiri adalah sejenis habitat yang di dalamnya banyak spesies bergantung. Oleh karena itu, aksi dari manusia dalam menyebabkan kebakaran hutan harus diadili. Terlebih jika itu dengan tujuan untuk kepentingan diri mereka sendiri. Ada banyak alasan yang dimiliki oleh oknum saat melakukan aksi pembakaran hutan, di antaranya adalah untuk pembukaan lahan yang baru atau pembangunan gedung-gedung yang baru dan lain-lain. Akan tetapi, mereka sama sekali tidak memikirkan bagaimana nasib dari flora dan juga fauna yang ada di dalam hutan tersebut. Flora serta fauna yang terdapat di dalam hutan tentu akan melarikan diri. Namun, tentu ada juga yang hangus terbakar api lantaran ulah dari manusia itu sendiri. Mereka tentu akan kehilangan tempat tinggal aslinya. Bahkan akan menjadi keresahan tersendiri jika mereka masuk ke wilayah pemukiman penduduk karena perasaan tidak memiliki rumah untuk tinggal. Maka tidak mengherankan jika akhir-akhir ini ada banyak kasus penemuan hewan liar seperti singa dan macan yang masuk ke pemukiman warga. Berbeda lagi dengan faktor alam misalnya karena kemarau panjang atau adanya sambaran petir kala hujan datang. Musim tentu tidak bisa diperkirakan oleh manusia, sehingga saat kemarau datang dengan masa yang amat panjang adalah hal yang sangat wajar. Akan tetapi, hal tersebut sangat berpengaruh kepada kondisi hutan yang setiap hari terkena sengat matahari menyebabkan munculnya percikan api. Juga karena adanya petir yang menyambar sehingga memunculkan percikan api."

judulnya : Kebakaran Hutan yang Terjadi di Indonesia
dari https://moondoggiesmusic.com/contoh-artikel/

In [15]:
cleaning_coba = get_clean_corpus(corpus=coba, stopwords=stopwords)

In [16]:
for item in cleaning_coba['dokumen']:
    print(item)

Indonesia adalah salah satu Negara yang di dalamnya memiliki iklim tropis.
Negara ini terdiri dari banyak pulau di setiap wilayahnya.
Meskipun harus diakui bahwa di Indonesia sendiri, dataran yang ada tentu tidak seluas lautan yang membentang luas.
Namun demikian, Indonesia memiliki kawasan hutan yang cukup banyak mulai dari Sabang yang terletak di provinsi Aceh sampai Merauke yang terletak di kawasan Papua.
Namun, di beberapa tahun terakhir, Indonesia sering mengalami kebakaran hutan lantaran beberapa faktor yang ada, mulai dari faktor buatan atau dari manusianya sendiri hingga faktor alam.
Faktor alam yang menyebabkan kebakaran hutan memang tidak bisa dihindarkan dan tidak ada yang bisa disalahkan dalam hal ini.
Akan tetapi, untuk faktor dari tindakan manusia perlu untuk ditindak dan dievaluasi.
Memang sebuah keresahan tersendiri dimana manusia banyak yang kini kehilangan kesadarannya sampai-sampai melakukan suatu perbuatan yang bisa merugikan banyak orang termasuk dirinya sendiri, k

In [17]:
tf_coba, df_idf_coba  = get_term_weighting_score(cleaning_result=cleaning_coba)

In [18]:
cobs = Graph(result_doc=cleaning_coba, raw_frequency=tf_coba, idf=df_idf_coba)
for item in cobs.summarize:
#     print(item.id)
    print(item.full_sentence)
#     print(item.pagerank_score)

Indonesia adalah salah satu Negara yang di dalamnya memiliki iklim tropis.
Namun, di beberapa tahun terakhir, Indonesia sering mengalami kebakaran hutan lantaran beberapa faktor yang ada, mulai dari faktor buatan atau dari manusianya sendiri hingga faktor alam.
Faktor alam yang menyebabkan kebakaran hutan memang tidak bisa dihindarkan dan tidak ada yang bisa disalahkan dalam hal ini.
Oleh karena itu, aksi dari manusia dalam menyebabkan kebakaran hutan harus diadili.
Ada banyak alasan yang dimiliki oleh oknum saat melakukan aksi pembakaran hutan, di antaranya adalah untuk pembukaan lahan yang baru atau pembangunan gedung-gedung yang baru dan lain-lain.
Namun, tentu ada juga yang hangus terbakar api lantaran ulah dari manusia itu sendiri.
Bahkan akan menjadi keresahan tersendiri jika mereka masuk ke wilayah pemukiman penduduk karena perasaan tidak memiliki rumah untuk tinggal.


In [19]:
for item in cobs.outlier:
    print(item.full_sentence)

Terlebih jika itu dengan tujuan untuk kepentingan diri mereka sendiri.


In [20]:
tf_coba

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
terms,,,,,,,,,,,,,,,,,,,,,
aceh,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adil,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
aksi,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
alam,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
alami,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
api,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
asa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
asli,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [21]:
df_idf_coba

,df_term,idf_term
terms,,
aceh,1,1.361728
adil,1,1.361728
aksi,2,1.060698
alam,3,0.884607
alami,1,1.361728
alas,1,1.361728
api,3,0.884607
asa,1,1.361728
asli,1,1.361728


In [22]:
a = [1,2,3,4]
a.pop(0)

1

In [23]:
a

[2, 3, 4]